In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('train.csv')
df.head()

,date,sr,fact,enc_hub
0,2021-12-03,1,2,78
1,2022-06-04,27,79,28
2,2021-03-22,9380,11002,104
3,2021-07-31,294,308,70
4,2022-06-04,2,306,92


In [9]:
df.date = df.date.astype(np.datetime64)
df = df.sort_values(by='date')
df

,date,sr,fact,enc_hub
6902,2021-01-02,302.0,6200.0,63
7017,2021-01-02,1.0,7.0,94
7000,2021-01-02,5.0,71.0,77
6994,2021-01-02,21.0,106.0,128
6967,2021-01-02,22.0,144.0,135
...,...,...,...,...
1639,2022-07-16,1.0,1.0,78
1590,2022-07-16,966.0,1189.0,79
1644,2022-07-16,12666.0,16252.0,47
1616,2022-07-16,3897.0,3751.0,24


In [11]:
df.enc_hub.unique()

array([ 63,  94,  77, 128, 135, 113,  53, 133, 121,  65,  72, 100,  90,
       104,  61,  18, 103,  64,  25,  62,  33,  97, 105,  31,  84,  50,
        15, 123,  38, 108, 118,  79, 115,  28,   6,   3, 132,  11, 126,
       112,  22,  34,  70,  16, 122,  88,  98,  93,  13,  29,  87,   5,
        37,  86,  71,  23, 125,  10, 101,  26, 110,  44, 116,  91,  68,
       114,  76,   8, 134,  89,  39,  17, 131,  43, 102, 130,   4,  14,
        57, 129,  73,  69,  35,  19,  95, 127, 111,  12, 117, 120,  99,
        46,  36,  75,  66,  85,   0,  47, 119, 107,  96,  41,  81,  30,
        32,  60,  58,  78,  92,  21,  49,  74,   1,  40,  27,   7,  55,
       136,  82, 124,  24,  51,  20,   9,  56,  80,   2,  45,  67, 109,
        83,  42,  48])

In [14]:
print(f"{df.enc_hub.min()}, {df.enc_hub.max()}")

0, 136


In [15]:
dates = pd.date_range(start=df.date.min(), end=df.date.max())
hubs = range(137)

In [26]:
df.set_index(['enc_hub', 'date'])

,,sr,fact
enc_hub,date,,
63,2021-01-02,302.0,6200.0
94,2021-01-02,1.0,7.0
77,2021-01-02,5.0,71.0
128,2021-01-02,21.0,106.0
135,2021-01-02,22.0,144.0
...,...,...,...
78,2022-07-16,1.0,1.0
79,2022-07-16,966.0,1189.0
47,2022-07-16,12666.0,16252.0


In [154]:
_df = df.set_index(['enc_hub', 'date']).reindex(pd.MultiIndex.from_product([hubs, 
dates], names=['hub', 'date']), fill_value=0)

In [155]:
_df

sr    fact
hub date                      
0   2021-01-02     0.0     0.0
    2021-01-03     0.0     0.0
    2021-01-04     0.0     0.0
    2021-01-05     0.0     0.0
    2021-01-06     0.0     0.0
...                ...     ...
136 2022-07-12  2174.0  2643.0
    2022-07-13  1490.0  1772.0
    2022-07-14  1199.0  1521.0
    2022-07-15  1480.0  1947.0
    2022-07-16   459.0   642.0

[76857 rows x 2 columns]

In [187]:
def prep_data(df, lag_counts:(int,int)):
#     df.date = df.date.astype(np.datetime64)
    df.fact = df.fact.astype(np.float64)
    df.sr = df.sr.astype(np.float64)
    df['y'] = df.groupby(level=0)['fact'].shift(-1) 
    
    for sr_lag in range(1, lag_counts[0]+1):
        df[f'sr_{sr_lag}'] = df.groupby(level=0)['fact'].shift(sr_lag)
    
    for fact_lag in range(1, lag_counts[1]+1):
        df[f'fact_{fact_lag}'] = df.groupby(level=0)['fact'].shift(fact_lag)
    
    return df.dropna().reset_index().sort_values(by='date').reset_index().drop(columns='index')

In [188]:
x = prep_data(_df, [2, 2])
x

,hub,date,sr,fact,y,sr_1,sr_2,fact_1,fact_2
0,0,2021-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,101,2021-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,64,2021-01-04,1692.0,5045.0,5205.0,1680.0,1660.0,1680.0,1660.0
3,73,2021-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,34,2021-01-04,11.0,44.0,2.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
76441,31,2022-07-15,3082.0,4008.0,2255.0,3725.0,4259.0,3725.0,4259.0
76442,103,2022-07-15,0.0,0.0,0.0,23.0,8714.0,23.0,8714.0
76443,32,2022-07-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76444,67,2022-07-15,14822.0,22513.0,13836.0,22598.0,23855.0,22598.0,23855.0


In [189]:
X = x.drop(columns=['date', 'y'])
y = x.y

In [190]:
x.groupby('date').count()

,hub,sr,fact,y,sr_1,sr_2,fact_1,fact_2
date,,,,,,,,
2021-01-04,137,137,137,137,137,137,137,137
2021-01-05,137,137,137,137,137,137,137,137
2021-01-06,137,137,137,137,137,137,137,137
2021-01-07,137,137,137,137,137,137,137,137
2021-01-08,137,137,137,137,137,137,137,137
...,...,...,...,...,...,...,...,...
2022-07-11,137,137,137,137,137,137,137,137
2022-07-12,137,137,137,137,137,137,137,137
2022-07-13,137,137,137,137,137,137,137,137


In [191]:
X.dtypes

hub         int64
sr        float64
fact      float64
sr_1      float64
sr_2      float64
fact_1    float64
fact_2    float64
dtype: object

In [201]:
X

,hub,sr,fact,sr_1,sr_2,fact_1,fact_2
0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,101,0.0,0.0,0.0,0.0,0.0,0.0
2,64,1692.0,5045.0,1680.0,1660.0,1680.0,1660.0
3,73,0.0,0.0,0.0,0.0,0.0,0.0
4,34,11.0,44.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
76441,31,3082.0,4008.0,3725.0,4259.0,3725.0,4259.0
76442,103,0.0,0.0,23.0,8714.0,23.0,8714.0
76443,32,0.0,0.0,0.0,0.0,0.0,0.0
76444,67,14822.0,22513.0,22598.0,23855.0,22598.0,23855.0


In [205]:
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error

def score(X, y, model):
    indices_dict = X.groupby('hub').indices
    hub_metrics = []
    for key, value in indices_dict.items():
        y_pred = model.predict(X.iloc[value])
        y_true = y.iloc[value]
        mae = mean_absolute_error(y_true, y_pred)
        mape = mean_absolute_percentage_error(y_true, y_pred)
        hub_metrics.append({
            'hub': key,
            'mae': mae,
            'mape': mape
        }
        )
    return hub_metrics

In [206]:
from sklearn.model_selection import TimeSeriesSplit

model = RandomForestRegressor(n_estimators=100)
tscv = TimeSeriesSplit(test_size=137*60)
records = []

for train_index, test_index in tscv.split(X):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    model.fit(X_train, y_train)
    records.extend(score(X_test, y_test, model))
#     break

results = pd.DataFrame.from_records(records)

In [210]:
results.groupby('hub').mean().sort_values('mae')

,mae,mape
hub,,
90,0.000000,0.000000e+00
115,0.000000,0.000000e+00
38,0.000000,0.000000e+00
3,0.000000,0.000000e+00
34,0.000000,0.000000e+00
...,...,...
63,2795.049700,6.101462e+16
104,3298.899133,2.027394e+01
62,4103.125200,2.078806e-01


In [93]:
from sklearn.ensemble import RandomForestRegressor

In [94]:
model = RandomForestRegressor(n_estimators=100, max_depth=4)

In [95]:
model.fit(X, y)

RandomForestRegressor(max_depth=4)

In [96]:
model.score(X, y)

0.9016887167242764

In [ ]:
for train_index, test_index in tscv.split(df):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]